In [1]:
import os
import shutil
from sklearn.model_selection import train_test_split

path_to_images_0 = '../datasets/data_fire/0'
path_to_images_1 = '../datasets/data_fire/1'

# Splitting dataset into training and test sets
images_0 = os.listdir(path_to_images_0)
images_1 = os.listdir(path_to_images_1)

train_images_0, test_images_0 = train_test_split(images_0, train_size=80, random_state=42)
train_images_1, test_images_1 = train_test_split(images_1, train_size=80, random_state=42)

# Directories for train and test
os.makedirs('dataset/train/0', exist_ok=True)
os.makedirs('dataset/train/1', exist_ok=True)
os.makedirs('dataset/test/0', exist_ok=True)
os.makedirs('dataset/test/1', exist_ok=True)

for img in train_images_0:
    shutil.copy(os.path.join(path_to_images_0, img), 'dataset/train/0/' + img)

for img in train_images_1:
    shutil.copy(os.path.join(path_to_images_1, img), 'dataset/train/1/' + img)

for img in test_images_0:
    shutil.copy(os.path.join(path_to_images_0, img), 'dataset/test/0/' + img)

for img in test_images_1:
    shutil.copy(os.path.join(path_to_images_1, img), 'dataset/test/1/' + img)


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    'dataset/test',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)


Found 160 images belonging to 2 classes.
Found 491 images belonging to 2 classes.


In [3]:
from tensorflow.keras import layers, models

model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
history = model.fit(
    train_generator,
    steps_per_epoch=50,
    epochs=30,
    validation_data=test_generator,
    validation_steps=25
)


Epoch 1/30


c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 5/50 ━━━━━━━━━━━━━━━━━━━━ 11s 248ms/step - accuracy: 0.4995 - loss: 0.8008

c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


50/50 ━━━━━━━━━━━━━━━━━━━━ 11s 133ms/step - accuracy: 0.4999 - loss: 0.8229 - val_accuracy: 0.5560 - val_loss: 0.6940
Epoch 2/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 80ms/step - accuracy: 0.6554 - loss: 0.6288 - val_accuracy: 0.7149 - val_loss: 0.7012
Epoch 3/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 78ms/step - accuracy: 0.7938 - loss: 0.5082 - val_accuracy: 0.8778 - val_loss: 0.3283
Epoch 4/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 76ms/step - accuracy: 0.7430 - loss: 0.5260 - val_accuracy: 0.7047 - val_loss: 0.6035
Epoch 5/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 77ms/step - accuracy: 0.7829 - loss: 0.4642 - val_accuracy: 0.8819 - val_loss: 0.2728
Epoch 6/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 78ms/step - accuracy: 0.8214 - loss: 0.4951 - val_accuracy: 0.7536 - val_loss: 0.5076
Epoch 7/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 80ms/step - accuracy: 0.7836 - loss: 0.4597 - val_accuracy: 0.8534 - val_loss: 0.2929
Epoch 8/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 5s 78ms/step - accuracy: 0.8031 - loss: 0.4395 - val_accuracy: 0.7821 - val_loss: 

In [5]:
test_loss, test_acc = model.evaluate(test_generator, steps=25)
print('Test accuracy:', test_acc)


25/25 ━━━━━━━━━━━━━━━━━━━━ 3s 114ms/step - accuracy: 0.8752 - loss: 0.2956
Test accuracy: 0.8961303234100342


In [6]:
model.save('../../models/fire_detection_model.h5')


In [9]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image as keras_image
from PIL import Image
import numpy as np

loaded_model = load_model('../../models/fire_detection_model.h5')

img_path = 'image_no_fire.jpg'
img = Image.open(img_path)

img = img.resize((150, 150))

img_array = np.array(img) / 255.0

img_array = np.expand_dims(img_array, axis=0)

prediction = loaded_model.predict(img_array)[0][0]

confidence = prediction * 100

if prediction > 0.5:
    result = "Fire"
else:
    result = "No fire"

print(f"Confidence that the image contains fire: {confidence:.2f}%")
print(f"Result: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Confidence that the image contains fire: 4.65%
Result: No fire
